In [1]:
import numpy as np
import json
import os
import re
import collections
from sklearn.datasets.base import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import codecs
import pickle
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText
import sys
from __future__ import division
from collections import OrderedDict
import pandas as pd
from sklearn.datasets import load_files
from collections import OrderedDict

In [2]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import KFold
from surprise import accuracy

In [3]:
def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [4]:
path = '/home/irlab/Documents/recsys/mpd/data'
filenames = os.listdir(path)
mapping = {}
playlist_prob = OrderedDict()

In [5]:
#Train FastText Model for finding similarity
def train_playlist_embedding(filenames):    
    data = LineSentence('/home/irlab/files/data.txt')
    model = FastText(size=100)
    print('Building Vocab....')
    model.build_vocab(data)
    print('Training....')
    model.train(data, total_examples=model.corpus_count, epochs=model.iter)
    print(model)
    return model

In [7]:
#print(train_data)
model = train_playlist_embedding(filenames)

Building Vocab....
Training....


/home/irlab/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


FastText(vocab=9132, size=100, alpha=0.025)


In [8]:
#Find Similar Playlists with given name
def find_similar(playlist_name,model = model):
    names = []
    similar_playlists = model.wv.similar_by_word(playlist_name,20)
    similar_playlists = dict(similar_playlists)
    sim_playlist_name = [key for (key,value) in sorted(similar_playlists.items(),key=lambda x:x[1],reverse=True)]
    return sim_playlist_name

In [9]:
def load_filemapping():
    fp = open('/home/irlab/files/mappings/track_filename.json')
    mpd_mapping = json.load(fp)
    return mpd_mapping

In [10]:
mpd_mapping = load_filemapping()

In [11]:
#Change this as per you files folder location
fp = open('/home/irlab/files/stats/album_stats.json')
album_map = json.load(fp)
album_stat = sum(album_map.values())

fp = open('/home/irlab/files/stats/artist_stats.json')
artist_map = json.load(fp)
artist_stat = sum(artist_map.values())

fp = open('/home/irlab/files/stats/tracks_stats.json')
track_map = json.load(fp)
track_stat = sum(track_map.values())

print(album_stat,artist_stat,track_stat)

(66346428, 66346428, 66346428)


In [12]:
#Generate weights for the tracks
def create_weighted_playlists(playlist_name,playlist_prob):
    try:
        similar_playlists = find_similar(playlist_name)
    except KeyError:
        temp = playlist_name.split()
        playlist_name = ''.join(temp)
        similar_playlists = find_similar(playlist_name)
        
    #print(similar_playlists)
    for similar_playlist in similar_playlists:
        if similar_playlist in mpd_mapping.keys():    
            #print(similar_playlist)
            name = mpd_mapping[similar_playlist]
            filename = os.sep.join((path,name))
            fp = open(filename,'r')
            playlist_file = json.load(fp)
            mpd_data = playlist_file['playlists']
            playlist_prob[similar_playlist] = {}
            for playlist in mpd_data:
                if normalize_name(playlist['name']) in similar_playlist:
                    for track in playlist['tracks']:
                        p1 = artist_map[track['artist_uri']]/artist_stat
                        p2 = album_map[track['album_uri']]/album_stat
                        p3 = track_map[track['track_uri']]/track_stat
                        playlist_prob[similar_playlist][track['track_uri']] = (p1+p2+p3)/(1+track['pos'])
                        #print(playlist_prob[similar_playlist])
    return playlist_prob

In [13]:
#Create Rating matrix
def create_matrix(playlist_prob):
    pd.DataFrame(data=playlist_prob,dtype=np.int64)
    df = pd.DataFrame(data=playlist_prob)
    df = df.fillna(0)
    df = df.T
    return df

In [14]:
#Load Data in surprise format
def load_data(tracks,playlists,ratings):
    ratings_dict = {'itemID': tracks,
                    'userID': playlists,
                    'rating': ratings}
    df1 = pd.DataFrame(ratings_dict)
    reader = Reader(rating_scale=(0,1))
    data = Dataset.load_from_df(df1[['userID','itemID','rating']],reader)
    #print(df1)
    return data

In [15]:
#Train using SVD and Gradient Descent
def train(data):
    kf = KFold(n_splits=12)
    algo = SVD(n_epochs=50,lr_all=0.005,reg_all=0.075)
    for trainset,testset in kf.split(data):
        algo.fit(trainset)
        prediction = algo.test(testset)
        print(accuracy.rmse(prediction,verbose=True))
    return algo

In [16]:
#Predict ratings for unknown tracks and filter them out
def test(playlist_name,tracks,algo,seed_tracks):
    tracks_pred = [n for n in tracks if n not in seed_tracks]
    Recommend = {}
    for track in tracks_pred:
        predict = algo.predict(playlist_name,track)
        Recommend[track] = predict[3]
    recommend_list = [key for (key,value) in sorted(Recommend.items(),key=lambda x:x[1],reverse=True)]
    return recommend_list

In [17]:
#If we fall short of 500 tracks pick popular tracks
def random_tracks(recommend_list,seed_tracks):
    tracks = [key for (key,value) in sorted(track_map.items(),key=lambda x:x[1],reverse=True)]
    extend_tracks = [track for track in tracks if track not in recommend_list]
    extend_tracks = [track for track in extend_tracks if track not in seed_tracks]
    diff = 500 - len(recommend_list)
    print(len(extend_tracks[0:diff]))
    for name in extend_tracks[0:diff]:
        print(name)
        recommend_list.append(name)
    return recommend_list

In [18]:
#change this path as per your system
path1 = '/home/irlab/Documents/recsys/challenge'
challenge_file = 'challenge_set.json'
filename = os.sep.join((path1,challenge_file))
fp = open(filename)
challenge_set = json.load(fp)
playlists = challenge_set['playlists']
dict_playlist = {}

In [19]:
import csv

In [20]:
#string = ['team_info','main','Dark_Horse_1','201711040@daiict.ac.in']

#with open('/home/irlab/Documents/recsys/mpd/codes/output/output2.csv','w') as csvfile:
#        filewriter = csv.writer(csvfile,delimiter = ',')
#        filewriter.writerow(string)

count=0
for playlist in playlists:
    list_temp = []
    seed_tracks = []
    recommend_list = []
    if 'name' in playlist.keys() and playlist['num_samples']==10:
        pid = playlist['pid']
        list_temp.insert(0,pid)
        count+=1
        print(count,playlist['name'])
        tracks = playlist['tracks']
        playlist_prob.clear()
        playlist_prob[normalize_name(playlist['name'])] = {}
        i=0
        for track in tracks:
            if track['artist_uri'] not in artist_map.keys():
                p1 = 1
            else:    
                p1 = artist_map[track['artist_uri']]/artist_stat
            if track['album_uri'] not in artist_map.keys():
                p2 = 1
            else:
                p2 = album_map[track['album_uri']]/album_stat
            if track['track_uri'] not in artist_map.keys():
                p3 = 1
            else:
                p3 = track_map[track['track_uri']]/track_stat
            seed_tracks.insert(i,track['track_uri'])
            playlist_prob[normalize_name(playlist['name'])][track['track_uri']] = (p1+p2+p3)/(1+track['pos']) 
            i+=1
        #print(seed_tracks)
        print('Finding Similar Playlists...')
        playlist_prob = create_weighted_playlists(normalize_name(playlist['name']),playlist_prob)
        print('Creating Rating matrix...')
        df = create_matrix(playlist_prob)
        column_names = list(df.columns)
        row_names = list(df.index)
        matrix = df.values
        row,col = np.nonzero(matrix)
        row = row.tolist()
        col = col.tolist()
        n_tracks = [column_names[i] for i in col]
        playlists = [row_names[i] for i in row]
        ratings = [matrix[r][c] for (r,c) in zip(row,col)]
        print('Loading data for training')
        data = load_data(n_tracks,playlists,ratings)
        print('Training....')
        algo = train(data)
        print('Predicting....')
        recommend_list = test(normalize_name(playlist['name']),n_tracks,algo,seed_tracks)
        print('Playlist Size:',len(recommend_list))
        if len(recommend_list[0:500])<500:
            print('Picking popular tracks...')
            recommend_list = random_tracks(recommend_list,seed_tracks)
            print('New Playlist Size:',len(recommend_list))
        list_temp.extend(recommend_list[0:500])
        print('Writing to file.....')
        #Change this path as per your system
        with open('/home/irlab/Documents/recsys/mpd/codes/output/final_submission.csv','a') as csvfile1:
            filewriter = csv.writer(csvfile1,delimiter = ',')
            filewriter.writerow(list_temp)

(1, u'Fix You')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0057
0.005650999614259824
RMSE: 0.0146
0.014562170499885677
RMSE: 0.0081
0.00811959985245429
RMSE: 0.0576
0.05755264713608919
RMSE: 0.0410
0.041006712092353695
RMSE: 0.0099
0.009914023443932808
RMSE: 0.0156
0.015640200126178068
RMSE: 0.0054
0.00536966772612854
RMSE: 0.1352
0.13524858980729035
RMSE: 0.0030
0.002976412041871736
RMSE: 0.0047
0.004677585962426612
RMSE: 0.0217
0.021702982088219264
Predicting....
('Playlist Size:', 1699)
Writing to file.....
(2, u'Relax')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0141
0.014115270855435964
RMSE: 0.0076
0.007551803782119711
RMSE: 0.0111
0.01105018060104452
RMSE: 0.0091
0.009050933660539981
RMSE: 0.0050
0.005038786123322742
RMSE: 0.0183
0.018271138392383212
RMSE: 0.0055
0.005535742690053979
RMSE: 0.0029
0.002906263735106673
RMSE: 0.0058
0.005809180068839295
RMSE: 0.1175

RMSE: 0.0173
0.01728745457860282
RMSE: 0.0055
0.005456842786698056
RMSE: 0.0195
0.019500505310903724
RMSE: 0.0230
0.02303211017301297
RMSE: 0.0031
0.0030894198771412315
RMSE: 0.1183
0.11830756504522576
RMSE: 0.0177
0.01765738750967889
RMSE: 0.0178
0.017807583942686632
RMSE: 0.0115
0.01149958793058413
RMSE: 0.0027
0.002715334543587744
RMSE: 0.0430
0.04296385927297167
RMSE: 0.0141
0.014120275359021769
Predicting....
('Playlist Size:', 2238)
Writing to file.....
(16, u'Introspective')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0012
0.0012159917339112033
RMSE: 0.0018
0.0017694328481417444
RMSE: 0.0081
0.008075570312854812
RMSE: 0.0031
0.003106977077382542
RMSE: 0.0027
0.0026772221592734724
RMSE: 0.1335
0.13349342246171167
RMSE: 0.0240
0.023959991445820672
RMSE: 0.0087
0.008686233500031695
RMSE: 0.0017
0.0017429702081256545
RMSE: 0.0103
0.01026777034749908
RMSE: 0.0498
0.04982652091739581
RMSE: 0.0253
0.02532410473700672
Predicting.

RMSE: 0.0121
0.012111702696430434
RMSE: 0.0035
0.003482073782170764
RMSE: 0.0027
0.0027215777617766845
RMSE: 0.0066
0.0065807235134739146
RMSE: 0.0060
0.006019767686911958
RMSE: 0.0064
0.006435772320244599
RMSE: 0.0031
0.0030963489464428456
RMSE: 0.1085
0.10846427838065324
RMSE: 0.0031
0.0031419517355577653
RMSE: 0.0096
0.009629027648740763
Predicting....
('Playlist Size:', 2725)
Writing to file.....
(30, u'Old Skool')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0075
0.007475007870567866
RMSE: 0.0182
0.01821002434101731
RMSE: 0.0191
0.019143474755089564
RMSE: 0.0126
0.012615475464641195
RMSE: 0.0047
0.0047005148544276626
RMSE: 0.0136
0.013619087465293058
RMSE: 0.0527
0.052668402587461985
RMSE: 0.0050
0.005027506564603564
RMSE: 0.0062
0.006223219799777394
RMSE: 0.0166
0.016647678469506065
RMSE: 0.0101
0.010087798705613897
RMSE: 0.1456
0.14555733548131605
Predicting....
('Playlist Size:', 1634)
Writing to file.....
(31, u'Dizzy')


RMSE: 0.0273
0.027292805278237624
RMSE: 0.0037
0.00374902781444165
RMSE: 0.1524
0.1523947467575477
RMSE: 0.0251
0.02508357266289157
RMSE: 0.0030
0.002970637388023261
RMSE: 0.0145
0.014459724157782154
RMSE: 0.0026
0.0025814298792917897
Predicting....
('Playlist Size:', 1466)
Writing to file.....
(44, u'Sandra')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0345
0.034460603555028384
RMSE: 0.0139
0.013853652267188724
RMSE: 0.0057
0.005724667712222382
RMSE: 0.0351
0.035147042246998196
RMSE: 0.0253
0.025258543444490675
RMSE: 0.0166
0.016565595558470838
RMSE: 0.2132
0.21316876747322128
RMSE: 0.0062
0.00620550317423725
RMSE: 0.0279
0.027913961002757268
RMSE: 0.0056
0.00563829016329326
RMSE: 0.0766
0.07658693412802788
RMSE: 0.0030
0.0030245747172949833
Predicting....
('Playlist Size:', 772)
Writing to file.....
(45, u'wild world')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0192
0.

RMSE: 0.0029
0.002882335678838609
RMSE: 0.0133
0.013301700285196167
RMSE: 0.0065
0.0064656816450828145
Predicting....
('Playlist Size:', 3589)
Writing to file.....
(58, u'Indie')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0087
0.008729932055078677
RMSE: 0.0095
0.009533720066048098
RMSE: 0.1284
0.128381632317184
RMSE: 0.0238
0.023825126885131248
RMSE: 0.0580
0.05799561822430545
RMSE: 0.0073
0.007324814514390582
RMSE: 0.0025
0.00246066202881872
RMSE: 0.0176
0.017643211835426456
RMSE: 0.0044
0.004390530325272111
RMSE: 0.0122
0.012244992642539022
RMSE: 0.0090
0.008957231321258585
RMSE: 0.0034
0.0034105552976371302
Predicting....
('Playlist Size:', 1846)
Writing to file.....
(59, u'WORKOUT :)')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0014
0.001391483135240943
RMSE: 0.0093
0.009326059354100765
RMSE: 0.0069
0.006874209716414751
RMSE: 0.0633
0.06327876966574093
RMSE: 0.0014


...

RMSE: 0.0681
0.06808199873766198
RMSE: 0.0107
0.010658058034485133
RMSE: 0.0179
0.0179176285136781
RMSE: 0.0034
0.0034078217351171426
RMSE: 0.0096
0.009597449559593052
RMSE: 0.0034
0.0034048243889891216
RMSE: 0.0164
0.016354417043914656
RMSE: 0.0222
0.022206750403896497
RMSE: 0.0280
0.027952536780063993
Predicting....
('Playlist Size:', 1595)
Writing to file.....
(943, u"Jake's playlist. ")
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0080
0.007954283091109795
RMSE: 0.0221
0.022066743931435137
RMSE: 0.0168
0.016816775782965778
RMSE: 0.0067
0.0067239729595244295
RMSE: 0.0054
0.005380861874003029
RMSE: 0.0397
0.03972086087645313
RMSE: 0.0127
0.012710086006780506
RMSE: 0.0969
0.09686512328173855
RMSE: 0.0187
0.018726006122413683
RMSE: 0.0065
0.006493037639214786
RMSE: 0.0225
0.022475690832000326
RMSE: 0.0136
0.013552400860289375
Predicting....
('Playlist Size:', 3024)
Writing to file.....
(944, u'Favorite')
Finding Similar Playlists

RMSE: 0.0049
0.004896748689914906
RMSE: 0.0111
0.011097712589326784
RMSE: 0.0155
0.015472183794795537
RMSE: 0.0115
0.011479276772234209
RMSE: 0.0076
0.007550005065834785
RMSE: 0.0153
0.015285992944331384
Predicting....
('Playlist Size:', 3316)
Writing to file.....
(957, u'Spotify & Chill')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0045
0.004491496662548281
RMSE: 0.0081
0.008114375370483895
RMSE: 0.0047
0.004721273026758575
RMSE: 0.0042
0.004221520989122211
RMSE: 0.0150
0.014954483447610705
RMSE: 0.0179
0.017891141347245433
RMSE: 0.0447
0.04466425095888911
RMSE: 0.1174
0.11738962913776713
RMSE: 0.0224
0.022392662352756266
RMSE: 0.0147
0.014661987667684414
RMSE: 0.0076
0.007628049731134662
RMSE: 0.0028
0.0028296690780166113
Predicting....
('Playlist Size:', 2370)
Writing to file.....
(958, u'21')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0075
0.007528306835208161
RMSE: 

RMSE: 0.0082
0.008159983386613152
RMSE: 0.0560
0.05601054496176083
Predicting....
('Playlist Size:', 5995)
Writing to file.....
(971, u'wedding')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0040
0.004016893106643925
RMSE: 0.0163
0.01625277312126219
RMSE: 0.0126
0.012603641493333522
RMSE: 0.0030
0.002987714885652928
RMSE: 0.0140
0.013979335893126887
RMSE: 0.0200
0.01996862829339273
RMSE: 0.0452
0.0451747649200361
RMSE: 0.0022
0.00215998752224841
RMSE: 0.1333
0.13334850423152184
RMSE: 0.0062
0.006215933258043056
RMSE: 0.0026
0.0025627961333006288
RMSE: 0.0026
0.0026020527217862845
Predicting....
('Playlist Size:', 2179)
Writing to file.....
(972, u'Favorite')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0112
0.011174011881002846
RMSE: 0.0066
0.0065935478655906815
RMSE: 0.0225
0.02251565629205605
RMSE: 0.0239
0.023864579435558843
RMSE: 0.0134
0.01342096479864012
RMSE: 0.0595


Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0247
0.024677046023922095
RMSE: 0.0165
0.0164850379272683
RMSE: 0.1224
0.12235695677750978
RMSE: 0.0186
0.01864768372738775
RMSE: 0.0061
0.006125406123168025
RMSE: 0.0057
0.005708188801264808
RMSE: 0.0172
0.01716996133853153
RMSE: 0.0475
0.047549294224652795
RMSE: 0.0047
0.004681163678946944
RMSE: 0.0394
0.03943810525692335
RMSE: 0.0138
0.013776293575438954
RMSE: 0.0046
0.00460415561392813
Predicting....
('Playlist Size:', 2062)
Writing to file.....
(986, u'hip')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0469
0.046871305152581554
RMSE: 0.0038
0.003847441037130586
RMSE: 0.0037
0.0036554933383380036
RMSE: 0.0130
0.01300248134918685
RMSE: 0.0096
0.009575094246689992
RMSE: 0.1171
0.11712558692270243
RMSE: 0.0248
0.024752587527661955
RMSE: 0.0199
0.019926672656547625
RMSE: 0.0166
0.01658876635246413
RMSE: 0.0039
0.00390014310143806

RMSE: 0.0291
0.02907608620916237
RMSE: 0.0136
0.013611664795543607
RMSE: 0.1711
0.17106641595087432
RMSE: 0.0066
0.006567060600120605
RMSE: 0.0063
0.006314024823620957
RMSE: 0.0230
0.023022409837652063
RMSE: 0.0217
0.021661370530651237
RMSE: 0.0115
0.011476464605235564
RMSE: 0.0106
0.01056248257563709
RMSE: 0.0344
0.03435533631082424
RMSE: 0.0247
0.024721959031972225
Predicting....
('Playlist Size:', 1047)
Writing to file.....
(1000, u'Classic Rock')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0100
0.00998313683993183
RMSE: 0.0433
0.04330840009099936
RMSE: 0.0036
0.003610815083096384
RMSE: 0.0051
0.005117952214912516
RMSE: 0.1000
0.10004979675304711
RMSE: 0.0031
0.003132336305699866
RMSE: 0.0074
0.00743601479088243
RMSE: 0.0197
0.019742813575402158
RMSE: 0.0102
0.01017836974287112
RMSE: 0.0240
0.023983253404478926
RMSE: 0.0215
0.02148886129685131
RMSE: 0.0070
0.007031943917440185
Predicting....
('Playlist Size:', 2727)
Writing t

In [81]:
#for playlist in playlists:
    #if 'name' in playlist.keys() and playlist['num_samples']==10:
           #print(playlist['name'])
playlist_prob = create_weighted_playlists('killin it',playlist_prob)
print('Creating Rating matrix...')
df = create_matrix(playlist_prob)

Creating Rating matrix...
